In [1]:
import csv
import plotly.graph_objects as go
import math
import plotly.express as px
import numpy as np
import pandas as pd
from IPython.display import display

# Load the data
file_path = 'Garmin_clean7.csv'
df = pd.read_csv(file_path)

# Convert "Avg.Pace" to string (if necessary)
df['Avg.Pace'] = df['Avg.Pace'].astype(str)
df['Avg.Pace'] = '0:' + df['Avg.Pace']

df['Avg Moving Pace'] = pd.to_timedelta(df['Avg.Pace']).dt.total_seconds()

df['Date'] = pd.to_datetime(df['Date'])

# Filter out the specific run
pace_to_remove = '0:10:52'     # Replace with the specific pace you want to remove
df = df[~((df['Avg.Pace'] == pace_to_remove))]

# Creates the ticks for the average paces
def get_pace_ylabels(ymin=360, ymax=720, step=30):
    y_ticks = np.arange(ymin, ymax, step)
    labels = [
        f"{'{:02d}'.format(math.floor(y/60))}:{'{:02d}'.format(int(y%60))}"
        for y in y_ticks
    ]
    return y_ticks, labels

fig = go.Figure()

# Add a trace for average pace with size representing distance
fig.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Avg Moving Pace'], 
    mode='markers', 
    marker=dict(
        size=df['Distance'],  # Assume 'Distance' is a column in your DataFrame
        sizemode='area',
        sizeref=2.*max(df['Distance'])/(40.**2),
        sizemin=4
    ),
    text=df['Distance'].apply(lambda d: f"{d} miles") + '<br>' + df['Date'].apply(lambda d: d.strftime('%m-%d-%Y')),
    hoverinfo='text',
    name='Avg Pace'
    
))

# Get custom y-ticks and labels
y_ticks, y_labels = get_pace_ylabels()

# Customize the layout
fig.update_layout(
    title='Fig 1: Average Pace Over Time (distance ~ radius)',
    xaxis_title='Date',
    yaxis_title='Avg Pace [min/mi]',
    xaxis=dict(
        tickformat='%b %Y',
        tickmode='linear',
        tick0=df['Date'].min(),
        dtick="M1"  # Ensure the tick labels show every month
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=y_ticks,
        ticktext=y_labels,
        autorange='reversed'  # Reverse the y-axis
    )
)
# Show the plot
fig.show()


In [2]:
# Get custom y-ticks and labels
fig_2 = go.Figure()

# Add a trace for average pace with size representing distance
fig_2.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Avg.HR'], 
    mode='markers', 
    marker=dict(
        size=df['Distance'],  # Assume 'Distance' is a column in your DataFrame
        sizemode='area',
        sizeref=2.*max(df['Distance'])/(40.**2),
        sizemin=4.
    ),
    text=df['Distance'].apply(lambda d: f"{d} miles\n") +'<br>' + 
    df['Avg.HR'].apply(lambda d: f"{d} BPM") + '<br>' + 
    df['Date'].apply(lambda d: d.strftime('%m-%d-%Y')),
    hoverinfo='text',
    name='Avg Pace'
    
))

# Get custom y-ticks and labels
y_ticks, y_labels = get_pace_ylabels()

# Customize the layout
fig_2.update_layout(
    title='Fig 2: Average heart rate per run over time (distance ~ radius)',
    xaxis_title='Date',
    yaxis_title='Avg heart rate [bpm]',
    xaxis=dict(
        tickformat='%b %Y',
        tickmode='linear',
        tick0=df['Date'].min(),
        dtick="M1"  # Ensure the tick labels show every month
    ),
    yaxis=dict(
        tickmode='array',
        ticktext=y_labels, # Assuming these labels are pre-calculated
        ticks='outside',   # This ensures tick marks appear outside the axis
        linecolor='black' # Color of the axis line
        
    )
)
# Show the plot
fig_2.show()

In [3]:
# Get custom y-ticks and labels
fig_3 = go.Figure()

# Add a trace for average pace with size representing distance
fig_3.add_trace(go.Scatter(
    x=df['Avg Moving Pace'], 
    y=df['Avg.HR'], 
    mode='markers', 
    marker=dict(
        size=df['Distance'],  # Assume 'Distance' is a column in your DataFrame
        sizemode='area',
         sizeref=2.*max(df['Distance'])/(40.**2),
        sizemin=4
    ),
    text=df['Distance'].apply(lambda d: f"{d} miles\n") + ' <br>'+ df['Avg.HR'].apply(lambda d: f"{d} BPM"),
    hoverinfo='text',
    name='Avg Pace'
    
))



# Get custom y-ticks and labels
x_ticks, x_labels = get_pace_ylabels()

# Customize the layout
fig_3.update_layout(
    title='Fig 3: Average heart rate over pace by run (distance ~ radius)',
    xaxis_title='Date',
    yaxis_title='Avg heart rate [bpm]',
    xaxis=dict(
        tickmode='array',
        tickvals=x_ticks,
        ticktext=x_labels,
        autorange='reversed'  # Reverse the y-axis
    ),
    yaxis=dict(
        tickmode='array',
        ticktext=y_labels, # Assuming these labels are pre-calculated
        ticks='outside',   # This ensures tick marks appear outside the axis
        linecolor='black' # Color of the axis line
        
    )
)
# Show the plot
fig_3.show()

In [4]:
#Adjustments
df = df.sort_values(by='Date')
#create a new cumalative sum column
df['Cumsum Distance'] = df['Distance'].cumsum()

fig_4 = go.Figure()
# Add trace for average pace with size representing distance
fig_4.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Cumsum Distance'],
    mode='lines',
    name='Cumsum Distance',
    text=df['Cumsum Distance'].apply(lambda d: f"{round(d, 2)} total miles") + '<br>' + df['Date'].apply(lambda d: d.strftime('%m-%d-%Y')) + '<br>' ,
    hoverinfo='text',
))


fig_4.update_layout(
    title='Fig 4: Cumalative distance over time',
    xaxis_title='Date',
    yaxis_title='Distance [mi]',
    xaxis=dict(
        tickformat='%b %Y',
        tickmode='linear',
        tick0=df['Date'].min(),
        dtick="M1"# show tick labels for every month
    )
    )

fig_4.show()

In [5]:
marathon_splits = ["9:16", "9:02", "8:56", "9:03", "8:58", "9:00", "8:58", "9:02", "9:25", "9:04", "8:43", "8:19", "8:36", "8:40", "8:38", "8:22", "8:34", "8:48", "8:49", "9:00", "8:54", "8:43", "8:43", "8:47", "8:58", "8:58"]
HR_splits = []

marathon_splits = ['0:0' + split for split in marathon_splits]

marathon_splits_Pandas = pd.Series(marathon_splits)
total_seconds = pd.to_timedelta(marathon_splits_Pandas).dt.total_seconds()

# Create the bar chart
fig_5 = go.Figure()



def get_pace_ylabels_2(ymin=360, ymax=540, step=30):
    y_ticks = np.arange(ymin, ymax, step)
    labels = [
        f"{'{:02d}'.format(math.floor(y/60))}:{'{:02d}'.format(int(y%60))}"
        for y in y_ticks
    ]
    return y_ticks, labels

# Get custom y-ticks and labels
y_ticks, y_labels = get_pace_ylabels_2()
miles = list(range(0, len(marathon_splits_Pandas)+1))



fig_5.add_trace(go.Bar(
    y = total_seconds-400,
    marker_color='skyblue',  # Optional: color of the bars
    name="Mile Splits",
    text=[f"Mile {i+1} <br> {marathon_splits_Pandas[i][3:]}" for i in range(len(marathon_splits_Pandas))], # Adding Mile labels
    hoverinfo='text',
))

# Add titles and labels
fig_5.update_layout(
    title="Marathon Mile Splits",
    xaxis_title="Mile",
    yaxis_title="1 Mile Split",
    yaxis=dict(
        showticklabels=False  # This hides the y-axis tick labels
    ),
    xaxis=dict(
        tickmode='array',
        tickvals = miles,
        ticktext=[f"{i+1}" for i in miles],  # Label each tick as "Mile 1", "Mile 2", etc.

    ),
    template="plotly_white"
    
)

# Show the chart
fig_5.show()



In [6]:
fig.write_html("graph1.html")

fig_2.write_html("graph2.html")

fig_3.write_html("graph3.html")

fig_4.write_html("graph4.html")

fig_5.write_html("graph5.html")

